In [1]:
import snappy

In [2]:
def cable_braid_word(companion_braid, p, q):
    """
    Compute the braid word for the (p,q)-cable of a knot given its companion braid word.
    
    Input:
    - companion_braid: list of integers representing the braid word of the companion knot
                      (positive integers for positive crossings, negative for negative)
    - p, q: integers representing the cabling parameters (p strands, q full twists)
    
    Output:
    - list of integers representing the braid word of the (p,q)-cable
    """
    from sage.groups.braid import BraidGroup
    
    # Determine the number of strands in the original braid
    n = max(abs(s) for s in companion_braid) + 1
    
    # Create the braid group for the companion knot
    B = BraidGroup(n)
    
    # Convert the companion braid word to a braid element
    companion_braid_elements = [B.gen(i-1) if i > 0 else B.gen(-i-1).inverse() for i in companion_braid]
    companion = B.one()
    for b in companion_braid_elements:
        companion *= b
    
    # Create the braid group for the cabled knot (has p*n strands)
    cabled_B = BraidGroup(p * n)
    
    # Initialize the cabled braid as the identity
    cabled_braid = cabled_B.one()
    
    # For each generator in the original braid, create its cabled version
    for s in companion_braid:
        original_strand = abs(s)
        is_positive = s > 0
        
        # The original generator affects strands original_strand-1 and original_strand
        # In the cabled version, this becomes p parallel strands
        start = (original_strand - 1) * p
        end = original_strand * p
        
        # Create the braid for this portion
        for i in range(start, end - 1):
            if is_positive:
                cabled_braid *= cabled_B.gen(i)
            else:
                cabled_braid *= cabled_B.gen(i).inverse()
    
    # Now add the torus knot braid (p,q) that links the cables
    # This is the full twist of p strands q times
    torus_part = cabled_B.one()
    for _ in range(abs(q)):
        # A full twist is (s_1 s_2 ... s_{p-1})^p
        for i in range(p - 1):
            if q > 0:
                torus_part *= cabled_B.gen(i)
            else:
                torus_part *= cabled_B.gen(i).inverse()
    
    # The cabled braid is the composition of the cabled companion and the torus part
    result_braid = cabled_braid * torus_part
    
    # Convert back to a list of integers
    result_word = []
    for term in result_braid.Tietze():
        result_word.append(term)
    
    return result_word

In [7]:
cable_braid_word([1], 2, 3)

[1, 1, 1, 1, 1, 1]

In [4]:
cable_braid_word([1,-2, 1, 2], 2, 1)

[1, -3, 1, 3, 1]

In [5]:
def cable_braid_word(companion_braid, p, q):
    """
    Compute the braid word for the (p,q)-cable of a knot given its companion braid word.
    
    Input:
    - companion_braid: list of integers representing the braid word of the companion knot
                      (positive integers for positive crossings, negative for negative)
    - p, q: integers representing the cabling parameters
    
    Output:
    - list of integers representing the braid word of the (p,q)-cable
    """
    from sage.groups.braid import BraidGroup
    
    # Determine the number of strands in the original braid
    n = max(abs(s) for s in companion_braid) + 1
    
    # Create the braid group for the companion knot
    B = BraidGroup(n)
    
    # Convert the companion braid word to a braid element
    companion = B.one()
    for s in companion_braid:
        if s > 0:
            companion *= B.gen(s-1)
        else:
            companion *= B.gen(-s-1).inverse()
    
    # Create the braid group for the cabled knot (has p strands)
    cabled_B = BraidGroup(p)
    
    # The cable consists of:
    # 1. The identity braid (parallel strands) conjugated by...
    # 2. The q-full-twist braid Δ^2q
    
    # Create the full twist (Δ^2) for p strands
    full_twist = cabled_B.one()
    for i in range(p-1):
        full_twist *= cabled_B.gen(i)
    for i in range(p-2, -1, -1):
        full_twist *= cabled_B.gen(i)
    
    # Create the q-full-twist
    torus_part = full_twist**q
    
    # The cabled braid word is the Tietze representation of the torus part
    cable_word = list(torus_part.Tietze())
    
    return cable_word